## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-02-08-26-55 +0000,wapo,Radioactive wasp nests found near nuclear stor...,https://www.washingtonpost.com/climate-environ...
1,2025-08-02-08-19-35 +0000,nyt,India Will Buy Russian Oil Despite Trump’s Thr...,https://www.nytimes.com/2025/08/02/world/asia/...
2,2025-08-02-08-00-00 +0000,cbc,Is the tradition of giving party leaders a fre...,https://www.cbc.ca/news/politics/leadership-co...
3,2025-08-02-08-00-00 +0000,cbc,The Eastern Energy Partnership: Atlantic Canad...,https://www.cbc.ca/news/politics/the-eastern-e...
4,2025-08-02-07-58-45 +0000,bbc,Car loan scandal payouts row - what's it about?,https://www.bbc.com/news/articles/c2l9vvj097lo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2304/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,trump,69
280,new,21
81,tariffs,17
13,russian,11
269,jobs,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
92,2025-08-01-22-33-00 +0000,wsj,President Trump fired the top Bureau of Labor ...,https://www.wsj.com/politics/policy/trump-orde...,145
101,2025-08-01-22-08-04 +0000,nypost,Trump fires Biden-appointed Bureau of Labor St...,https://nypost.com/2025/08/01/us-news/trump-fi...,127
224,2025-08-01-15-15-26 +0000,wapo,Trump’s new tariffs will hit these major tradi...,https://www.washingtonpost.com/business/2025/0...,123
121,2025-08-01-21-26-41 +0000,latimes,"A raging Trump digs in on his trade war, fires...",https://www.latimes.com/world-nation/story/202...,122
96,2025-08-01-22-27-28 +0000,nypost,Trump admin fires back at claims Clinton plan ...,https://nypost.com/2025/08/01/us-news/trump-ad...,121


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
92,145,2025-08-01-22-33-00 +0000,wsj,President Trump fired the top Bureau of Labor ...,https://www.wsj.com/politics/policy/trump-orde...
96,52,2025-08-01-22-27-28 +0000,nypost,Trump admin fires back at claims Clinton plan ...,https://nypost.com/2025/08/01/us-news/trump-ad...
133,52,2025-08-01-20-59-00 +0000,wsj,"Ghislaine Maxwell, the former associate of con...",https://www.wsj.com/politics/policy/epstein-as...
53,48,2025-08-02-00-59-00 +0000,wsj,The New York Fed’s John Williams says the labo...,https://www.wsj.com/economy/central-banking/fe...
218,47,2025-08-01-15-34-42 +0000,nypost,"Dow drops over 500 points as Trump tariffs, jo...",https://nypost.com/2025/08/01/business/dow-dro...
106,46,2025-08-01-21-57-00 +0000,wsj,The Trump administration is blocking funding f...,https://www.wsj.com/politics/policy/trump-cdc-...
213,43,2025-08-01-15-57-21 +0000,cbc,Viable trade deal wasn't on the table ahead of...,https://www.cbc.ca/news/politics/reaction-trum...
257,42,2025-08-01-11-25-01 +0000,nypost,Russian attack on Kyiv leaves at least 28 dead...,https://nypost.com/2025/08/01/world-news/russi...
172,40,2025-08-01-18-48-42 +0000,nypost,Elon Musk’s Tesla ordered to pay $243M in dead...,https://nypost.com/2025/08/01/business/elon-mu...
238,34,2025-08-01-14-08-00 +0000,wsj,"U.S. Envoy Steve Witkoff Visits Gaza as Trump,...",https://www.wsj.com/world/middle-east/u-s-envo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
